In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
# import torch

In [2]:
alert_df=pd.read_json("/Users/chrisgilibert/neuefische/Capstone-WIMT/data/alerts.json")

In [11]:
corazon_df= pd.read_json("/Users/chrisgilibert/neuefische/Capstone-WIMT/data/notification-labels-to-alert-surrogate-ids.json")

In [12]:
notification_df=pd.read_json("/Users/chrisgilibert/neuefische/Capstone-WIMT/data/notifications.json")

Alert df cleaning 

In [5]:
alert2= alert_df[['surrogate_id','agency']].set_index('surrogate_id')
alert2= pd.get_dummies(alert2)

In [7]:
grouped_alert= alert2.groupby(by='surrogate_id').sum()
grouped_alert.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8564 entries, 0A1RnzNX6xGq3mN3fZHpTQ to wPyUx-1W6xGm3_VmQsQ_lQ
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   agency_GewRJAw5tUmC4Ku4AX1-SQ  8564 non-null   uint8
 1   agency_GtvOEQAFZ0GtU6u4AXwvPg  8564 non-null   uint8
 2   agency_HE59N3RXM0q5vKu4AXlQZg  8564 non-null   uint8
 3   agency_JUR9bFXmVkWDHqu4AXaY0g  8564 non-null   uint8
 4   agency_JfA8Bw8Zp024Kqu4AXiSpQ  8564 non-null   uint8
 5   agency_MgUq5b9mOEunx6u4AXt_BA  8564 non-null   uint8
 6   agency_NuuRQ2I1Q0a50Kv-AVKlLA  8564 non-null   uint8
 7   agency_V2AIQQKgmUO3VazvAOA-Cw  8564 non-null   uint8
 8   agency_jLjibFoim0iwWau4AWoEdQ  8564 non-null   uint8
 9   agency_pky7jovXYkaw-awAAMrQ3g  8564 non-null   uint8
 10  agency_zCy9zG00HEqGeKu4AWZYNQ  8564 non-null   uint8
dtypes: uint8(11)
memory usage: 158.9+ KB


In [8]:
alert_df.drop(['agency'], axis=1, inplace=True)
alert_df.drop_duplicates(subset='surrogate_id',inplace=True,ignore_index=True)
alert_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8564 entries, 0 to 8563
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   document_id                 8564 non-null   object             
 1   surrogate_id                8564 non-null   object             
 2   created_at                  8564 non-null   datetime64[ns, UTC]
 3   published_at                8522 non-null   datetime64[ns, UTC]
 4   closed_at                   8511 non-null   datetime64[ns, UTC]
 5   cause                       8564 non-null   object             
 6   effect                      8564 non-null   object             
 7   description                 8564 non-null   object             
 8   is_global                   8564 non-null   object             
 9   area_of_effect_coordinates  8564 non-null   object             
dtypes: datetime64[ns, UTC](3), object(7)
memory usage: 669.2+ KB

In [9]:
alert_df= pd.merge(alert_df,grouped_alert, on='surrogate_id' )
alert_df

,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,area_of_effect_coordinates,...,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ
0,-MUH7_dRaJMkhnxY9bHs,0BBrcVJ26xGCgpvcY4zoVw,2021-02-24 03:43:22+00:00,2021-02-24 03:58:39+00:00,2021-02-24 04:21:00+00:00,Fire,Delays,🔥 Considera retrasos por servicios de emergenc...,false,"[{'latitude': -99.13399344738725, 'longitude':...",...,1,0,0,1,0,0,0,0,0,0
1,-MX9U6P85DXAwGIaDsgi,kIdXsXOS6xGH7OsWJh6kOw,2021-03-31 22:51:55+00:00,2021-03-31 22:54:19+00:00,2021-04-01 00:01:01+00:00,Fire,TrafficImpact,🔥 ¡Anticípate! Afectación vial por servicios d...,false,"[{'latitude': -99.13838681335923, 'longitude':...",...,1,0,0,0,0,1,0,0,0,1
2,-MafIvkWquSi6Yl2vmQd,EAkm7oW-6xGbNxdbuCW7Sw,2021-05-27 00:53:20+00:00,2021-05-27 00:53:24+00:00,2021-05-27 01:45:01+00:00,Fire,Delays,🔥 ¡Anticípate! Retrasos por incendio en Avenid...,false,"[{'latitude': -99.16168964039319, 'longitude':...",...,1,0,0,0,0,1,0,0,0,1
3,-MTwedXSvSQh9zB_GDfy,4Ntq2wtz6xGYvBd6hIEVJg,2021-02-19 23:40:32+00:00,2021-02-19 23:40:39+00:00,2021-02-20 00:23:00+00:00,Fire,Delays,⏳ Toma en cuenta servicios de emergencia atend...,false,"[{'latitude': -99.16983985780149, 'longitude':...",...,1,0,0,0,0,0,0,0,0,1
4,-M_1F3Xk4qFNXRvDyGd_,AJuYAn6u6xGNPhm2SVXXLA,2021-05-06 15:16:19+00:00,2021-05-06 15:16:39+00:00,2021-05-06 16:11:01+00:00,Fire,Delays,⏳ ¡Que no te agarre en curva! Hay retrasos por...,false,"[{'latitude': -99.16576423569903, 'longitude':...",...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8559,-MZZdjyfbnztCccgqoa1,sEfkzvmp6xG9Us96D1NGhQ,2021-04-30 21:19:54+00:00,2021-04-30 21:32:24+00:00,2021-04-30 22:14:00+00:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Oh que la! Afectación vial por semáforos de...,false,"[{'latitude': -99.15358161404815, 'longitude':...",...,0,0,0,0,0,0,0,0,0,1
8560,-MZor-DYgDMER9B2NtGD,QHbP5XKs6xGbFYutnY_XdQ,2021-05-04 00:51:44+00:00,2021-05-04 00:52:21+00:00,2021-05-04 01:45:01+00:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Nos lleva el chanfle! Afectación vial por s...,false,"[{'latitude': -99.11364770788302, 'longitude':...",...,0,0,0,0,0,0,0,0,0,1
8561,-MWizRKZ_YIPcMWzaqMP,QNJvrEGO6xGgXg3PRUP4fg,2021-03-26 14:43:48+00:00,2021-03-26 14:44:15+00:00,2021-03-26 16:01:01+00:00,BrokenTrafficLight,TrafficImpact,⏳ ¡No te esponjes! Hay retrasos por semáforos ...,false,"[{'latitude': -98.96204243026706, 'longitude':...",...,0,0,0,0,0,0,0,0,0,1
8562,-MS3yFJ5NoOacwMyGxSs,kO_VUshg6xGXlUHhke6ddw,2021-01-27 17:51:46+00:00,2021-01-27 17:51:51+00:00,2021-01-27 19:01:00+00:00,BrokenTrafficLight,Delays,⏳ ¡Chin! Hay retrasos por semáforos descompues...,false,"[{'latitude': -99.17786568559703, 'longitude':...",...,0,0,0,0,0,0,0,0,0,1


Merging DF

In [24]:
groupby_corazon= corazon_df.groupby(by="corazon_surrogate_id").count()
groupby_corazon

,day,notification_label_id
corazon_surrogate_id,,
0A1QcQE76xGgFcekx5UVQw,43,43
0A1RnzNX6xGq3mN3fZHpTQ,1,1
0A1mXq686xGBE1UqCdicFw,12,12
0A1yEQpS6xGuO50uuLA5sA,1,1
0A3No-Ii6xGz49sbt3N9VQ,4,4
...,...,...
wPsC4-t16xG4lBlKTuPaog,14,14
wPxAxXaN6xGtKmWjl4R2HA,14,14
wPxSTpQq6xGqzXE4eMMt0g,6,6


In [ ]:
alert_corazon = pd.merge(alert_df,corazon_df, left_on='surrogate_id', right_on='corazon_surrogate_id', how='left')

In [11]:
cause_effect_table= pd.pivot_table(alert_df, index=['cause'], columns='effect',aggfunc='count',fill_value=0,values="is_global", margins=True, margins_name= 'Total')

In [12]:
cause_effect_table

effect,,CirculationRestored,CirculationShutdown,Delays,Evacuation,FullCapacity,HighWaitingTime,InterimService,LaneReduction,RouteDetour,SecuritySpeed,SuspensionOfService,TrafficImpact,Total
cause,,,,,,,,,,,,,,
BrokenTrafficLight,0,0,0,23,0,0,0,0,2,0,0,0,22,47
BrokenVehicle,0,0,4,390,0,0,0,0,39,0,0,12,55,500
COVID19,0,0,7,1,0,5,0,0,0,0,0,18,1,32
Counterflow,0,0,0,2,0,0,0,0,0,0,0,0,2,4
CycleRide,0,0,4,3,0,0,0,0,0,0,0,0,16,23
Demonstration,1,321,1896,166,0,0,0,65,54,5,6,251,226,2991
EmergencyServices,2,1,20,860,0,0,0,0,208,0,0,4,253,1348
Event,0,2,4,0,0,0,0,0,2,0,3,5,0,16
Explosion,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [107]:
#!pip3 install transformers --no-input

from transformers import pipeline

nlp_pos_syntax = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-pos-syntax",
    tokenizer="mrm8488/bert-spanish-cased-finetuned-pos-syntax"
)

text = 'Considera retrasos por servicios de emergencia atendiendo incendio en Mesones y Av. 20 de Noviembre.'

nlp_pos_syntax(text)[1:len(nlp_pos_syntax(text))-1]

Dropping the rows that contain source in join_field

In [13]:
notification_df["join_field"].value_counts()

label      2203067
source       38591
alertId      36493
Name: join_field, dtype: int64

In [14]:
source_purge= notification_df[(notification_df["join_field"]=="source")].index
notification_df.drop(source_purge, axis=0, inplace=True)

Fixing the event_name column:

In [15]:
notification_df["event_name"].replace("notification_receive","notification_received", inplace=True)

In [16]:
notification_df["event_name"].replace("notification_open","notification_opened", inplace=True)

In [17]:
merged_df= pd.merge(notification_df, alert_df, left_on="join_key_value", right_on="document_id", how="outer")

In [18]:
print (notification_df.shape)
print (alert_df.shape)
print (merged_df.shape)

(2239560, 6)
(13542, 11)
(2305578, 17)


In [19]:
corazon_df

,day,corazon_surrogate_id,notification_label_id
0,2020-12-11 06:00:00 UTC,ILXZRxI86xGr4sWpUpBiwQ,FAQZVhI86xGr4sWpUpBiwQ
1,2020-12-12 06:00:00 UTC,QPvDTqE86xGB18lnrjsyMg,cfxfVKE86xGB18lnrjsyMg
2,2020-11-05 06:00:00 UTC,MIp_iZQf6xGvT1FRo8mp4A,1P2GjJQf6xGvT1FRo8mp4A
3,2020-11-05 06:00:00 UTC,cBvxJowf6xGvT1FRo8mp4A,Q1JpKowf6xGvT1FRo8mp4A
4,2020-11-05 06:00:00 UTC,kAsi5GAf6xGGvGubJnOcaQ,sFRt6WAf6xGGvGubJnOcaQ
...,...,...,...
166614,2021-02-17 06:00:00 UTC,UMzK_DBx6xG3U-XjxJ_HhA,23k4BzFx6xG3U-XjxJ_HhA
166615,2021-02-17 06:00:00 UTC,UMzK_DBx6xG3U-XjxJ_HhA,5KA4BzFx6xG3U-XjxJ_HhA
166616,2021-02-17 06:00:00 UTC,UMzK_DBx6xG3U-XjxJ_HhA,8Hk4BzFx6xG3U-XjxJ_HhA
166617,2021-02-17 06:00:00 UTC,8J0ZPTBx6xG3U-XjxJ_HhA,siVCSjBx6xG3U-XjxJ_HhA


    IMPORTANT COMBO

    TESTING SHIT OUT

In [ ]:
label_merger= notification_df.query("join_field == 'label'")
alert_id_merger= notification_df.query("join_field == 'alertId'")
merged_df_1= pd.merge(label_merger, alert_df, left_on="join_key_value", right_on="document_id", how="outer")
merged_df_2= pd.merge(alert_id_merger, alert_df, left_on="join_key_value", right_on="document_id", how="outer")

In [25]:
notification_df["join_key_value"].value_counts()

EP0z1BdU6xGPtZWUFIAO1g    2500
0CBmRWBX6xGxF_PRISPY8A    2291
MF0z7fFU6xGgIjsMRWjjiw    2189
EGUsS-Vy6xGrubm8lL1smw    2104
8OTqkB1U6xGPtZWUFIAO1g    2098
                          ... 
UpHzRSe36xGIPLtuPsYQjA       1
JjwesB6I6xG1RBe75o5jBw       1
Fb4iKP-z6xGe7StYX06yUA       1
EWt_e_J86xGgnGcJKC7IPw       1
6B5xvRXZ6xGAyyuyVTmASw       1
Name: join_key_value, Length: 93103, dtype: int64

In [24]:
notification_corazon=pd.concat([notification_df['join_key_value'] , corazon_df['corazon_surrogate_id']], axis =1)

notification_corazon

,join_key_value,corazon_surrogate_id
0,-Maph6wn8GA3SjPIc9HA,ILXZRxI86xGr4sWpUpBiwQ
1,-Maq63vqDUxQRawQ_3Au,QPvDTqE86xGB18lnrjsyMg
2,-Maq7-D7bhcEaFBU1LDW,MIp_iZQf6xGvT1FRo8mp4A
3,-MaqEsbKAqlxGG-RupjR,cBvxJowf6xGvT1FRo8mp4A
4,-MasFHCt_lTuNt_x16dH,kAsi5GAf6xGGvGubJnOcaQ
...,...,...
2278146,0NbaiCRr6xGFd2035-Gdlw,NaN
2278147,EMDjKLFr6xGRbrsa0gDjhg,NaN
2278148,YE2VDMZr6xGRbrsa0gDjhg,NaN
2278149,YBu-7NVr6xGRbrsa0gDjhg,NaN


In [ ]:
df1.merge(df2,left_on='col_name1', right_on='col_name2').merge(df3,left_on='col_name1', right_on='col_name3').drop(columns=['col_name2', 'col_name3']).rename(columns={'col_name1':'col_name'})

In [45]:
alert_df['surrogate_id'].value_counts()

UOaLuRlQ6xGI_zcOLODJsw    5
cM3gdRmw6xGzju-Q6nISag    5
sKHNmGTI6xGtWduDI3goEA    4
EOfffEZf6xGLXO0Oq8naUg    4
0GO5yEi06xGz_YW87SnlaA    4
                         ..
EL2a6yus6xGkSau8R0QT1w    1
oGNN0Wmz6xGy9r-REgGi4g    1
oB46ZipX6xGq3mN3fZHpTQ    1
EPLSl5Bv6xG4mAu7TKompg    1
sNtuasG06xGcVefmGOfyKQ    1
Name: surrogate_id, Length: 8564, dtype: int64

In [44]:
corazon_df['corazon_surrogate_id'].value_counts()

4I0uv5Zh6xGsJA3WI6GuGA    446
AN2aAvJf6xGzZ7VTtgSpyw    381
YEr3JDGO6xG9iQECLda_Aw    349
UFQNPLyQ6xGpXh-ihbTeHw    309
QH-eadBc6xGdn39znfuG8g    309
                         ... 
QMMOFBOs6xGsOFVTreCaAQ      1
YAzCi4gv6xG3Y3mJHBxlQA      1
8H-QBBRU6xG2XC35Amblxg      1
IFLxqOpm6xGFpE-1PiXznA      1
8BEghCaW6xGmFLu1O86fgw      1
Name: corazon_surrogate_id, Length: 11287, dtype: int64

In [118]:
corazon_df.shape

(166619, 3)

In [84]:
notification_df['join_key_value'].isin(alert_df['document_id']).value_counts()

False    2113465
True      126095
Name: join_key_value, dtype: int64

In [101]:
notification_df['join_field'].value_counts()

label      2203067
alertId      36493
Name: join_field, dtype: int64

In [87]:
notification_df['join_key_value'].isin(corazon_df['notification_label_id']).value_counts()

True     2100518
False     139042
Name: join_key_value, dtype: int64

In [76]:
notification_df['join_key_value'].isin(alert_df['document_id']).value_counts()

False    2113465
True      126095
Name: join_key_value, dtype: int64

In [77]:
notification_df['join_key_value'].isin(alert_df['surrogate_id']).value_counts()

False    2239560
Name: join_key_value, dtype: int64

In [ ]:
merged_df.head()

TESTS FROM BEFORE

In [66]:
#label_merger.where(label_merger["join_key_value"] == alert_df["surrogate_id"],True, False)
notification_df['join_key_value'].isin(alert_df['document_id']).value_counts()

#dfAns = df1.merge(df2, on = 'id', how = 'outer').merge(df3, on='id', how='outer').fillna(0)

False    2113465
True      126095
Name: join_key_value, dtype: int64

In [67]:
notification_df['join_key_value'].isin(alert_df['surrogate_id']).value_counts()

False    2239560
Name: join_key_value, dtype: int64